## <b><font color='darkblue'>Introduction</font></b>
([article source](https://www.linkedin.com/pulse/creating-your-own-llm-agent-from-scratch-step-by-step-avinash-dubey-lqajc/)) <b><font size='3ptx'>Large Language Models (LLMs) are powerful neural networks trained on massive amounts of text data.</font></b>

While they are capable of generating text, translating languages, and answering questions, they might not be suited for performing specific tasks. This is where LLM agents come into play. <b>LLM agents are programs that use LLMs to decide how and when to use tools to complete tasks. This guide will walk you through creating your own LLM agent from scratch</b>.

### <b><font color='darkgreen'>What are Agents?</font></b>
<font size='3ptx'><b>LLM agents are programs that utilize large language models to decide how and when to use tools to complete tasks</b></font>. <b>The core idea of agents is to use a language model to choose a sequence of actions to take</b>. In chains, a sequence of actions is hardcoded in code. In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

### <b><font color='darkgreen'>Why are LLM Agents Essential for Tasks?</font></b>
While LLMs are powerful and can generate text, translate languages, write different kinds of creative content, and answer questions, they may not be able to perform certain tasks. LLM agents provide a mechanism to extend the capabilities of LLMs by using them as reasoning engines to decide which actions to take and in what order.

### <b><font color='darkgreen'>Prerequisites</font></b>
To create your own LLM agent, you will need:
1. A base LLM
2. A tool that you will be interacting with (e.g., Google Custom Search Engine, Calculator)
3. An agent to control the interaction

## <b><font color='darkblue'>Agent Implementation</font></b>
First, install the required packages:

In [1]:
!pip install -q openai py_expression_eval google-api-python-client


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


### <b><font color='darkgreen'>Setting Up</font></b>
1. Set up a Custom Search Engine and get a Google API Key.
2. Set the API Key and Custom Search Engine ID as environment variables:

In [9]:
from openai import OpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from googleapiclient.discovery import build
from py_expression_eval import Parser
from dotenv import load_dotenv
import re, time, os

load_dotenv()

GOOGLE_CSE_ID = os.getenv('GOOGLE_CSE_ID')
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_SEARCH_API_KEY = os.getenv('GOOGLE_SEARCH_API_KEY')
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)

In [4]:
def search(search_term, search_api_key=GOOGLE_SEARCH_API_KEY, cse_id=GOOGLE_CSE_ID):
    search_result = ""
    service = build("customsearch", "v1", developerKey=search_api_key)
    res = service.cse().list(q=search_term, cx=cse_id, num=10).execute()
    
    for result in res['items']:
        search_result = search_result + result['snippet']
        
    return search_result 

In [5]:
parser = Parser()

def calculator(str):
    return parser.parse(str).evaluate({}) 

### <b><font color='darkgreen'>System Prompt</font></b>
This prompt instructs the model on how to interact with the user. It defines the available tools and the format to use them.

In [6]:
system_prompt = """

Answer the following questions and obey the following commands as best you can.

You have access to the following tools:

Search: useful for when you need to answer questions about current events. You should ask targeted questions.

Calculator: Useful for when you need to answer questions about math. Use python code, eg: 2 + 2

Response To Human: When you need to respond to the human you are talking to.

You will receive a message from the human, then you should start a loop and do one of two things

Option 1: You use a tool to answer the question.

For this, you should use the following format:

Thought: you should always think about what to do

Action: the action to take, should be one of [Search, Calculator]

Action Input: "the input to the action, to be sent to the tool"

After this, the human will respond with an observation, and you will continue.

Option 2: You respond to the human.

For this, you should use the following format:

Action: Response To Human

Action Input: "your response to the human, summarizing what you did and what you learned"

Begin!
"""

### <b><font color='darkgreen'>Stream Agent Function</font></b>

In [10]:
 def stream_agent(prompt):
    messages = [
        { "role": "system", "content": system_prompt },
        { "role": "user", "content": prompt },
    ]
    def extract_action_and_input(text):
          action_pattern = r"Action: (.+?)\n"
          input_pattern = r"Action Input: \"(.+?)\""
          action = re.findall(action_pattern, text)
          action_input = re.findall(input_pattern, text)
          return action, action_input
        
    while True:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            temperature=0,
            top_p=1,)
        
        response_text = response.choices[0].message.content
        print(response_text)
        #To prevent the Rate Limit error for free-tier users, we need to decrease the number of requests/minute.
        time.sleep(20)
        action, action_input = extract_action_and_input(response_text)
        if action[-1] == "Search":
            tool = search
        elif action[-1] == "Calculator":
            tool = calculator
        elif action[-1] == "Response To Human":
            print(f"Response: {action_input[-1]}")
            break
        observation = tool(action_input[-1])
        print("Observation: ", observation)
        messages.extend([
            { "role": "system", "content": response_text },
            { "role": "user", "content": f"Observation: {observation}" },
            ])

In [11]:
stream_agent("who is new openai board members")

Thought: I need to find the current board members of OpenAI. I will use the search tool for this.

Action: Search

Action Input: "New OpenAI board members"
Observation:  Mar 8, 2024 ... Dr. Sue Desmond-Hellmann, Nicole Seligman, Fidji Simo join; Sam Altman rejoins board.Mar 11, 2024 ... Who are OpenAI's new board members? · FIDJI SIMO · SUE DESMOND-HELLMAN · NICOLE SELIGMAN · Read Next · Technology · Site Index · Latest · Media.Mar 11, 2024 ... In a statement, OpenAI said Altman will rejoin the board alongside three new independent directors: Sue Desmond-Hellmann, former CEO of the Bill ...Nov 30, 2023 ... OpenAI Board Confirmed with Final Addition of Microsoft. In a new blog post, OpenAI and Sam Altman have confirmed its new look “initial” board ...Nov 22, 2023 ... Here are the newest members of OpenAI's board · Bret Taylor, board chair · Larry Summers · Adam D'Angelo.Nov 20, 2023 ... The center's founding director is Jason Gaverick Matheny, former director of IARPA. Its current execu